<a href="https://colab.research.google.com/github/kemwe/LLM/blob/master/implemenation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nvcc4jupyter
!pip install --upgrade diffusers
!pip install transformers accelerate safetensors

%load_ext nvcc4jupyter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 33.7 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.30.3
    Uninstalling diffusers-0.30.3:
      Successfully uninstalled diffusers-0.30.3
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp95qoa0gn".


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch



In [3]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}") # Print the device being used

Using device: cuda


In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [6]:
messages = [
   {"role": "user", "content": "How do chat templates work?"},
   {"role": "assistant", "content": "Chat templates help  LLMs like me generate more coherent responses by providing a structured way to organize the conversation."},
   {"role": "user", "content": "How do I use them?"},
]